In [1]:
import argparse
from nilearn.connectome import ConnectivityMeasure
from brainspace.gradient import GradientMaps
from brainspace.utils.parcellation import map_to_labels
import numpy as np
import nibabel as nib
from nilearn import datasets
import os.path as op
import os
from nilearn import signal
import pandas as pd
from scipy.sparse.csgraph import connected_components
from utils import get_basic_mask, cleanTS #saveGradToNPFile, npFileTofs5Gii,fsav5Tofsnative


240613-11:09:33,730 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.8.3


In [8]:
bids_folder='/Volumes/mrenkeED/data/ds-dnumrisk'
sub='01'
remove_task_effects=True
ses=1


fmriprep_confounds_include = ['global_signal', 'dvars', 'framewise_displacement', 'trans_x',
                                'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z',
                                'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02', 'a_comp_cor_03', 'cosine00', 'cosine01', 'cosine02'
                                ]



In [9]:
space = 'fsaverage5'
task = 'magjudge'
ru=1

timeseries = [None] * 2
for i, hemi in enumerate(['L', 'R']):
    filename = op.join(bids_folder,'derivatives', 'fmriprep', f'sub-{sub}', f'ses-{ses}', 'func', 
    f'sub-{sub}_ses-{ses}_task-{task}_run-{run}_space-{space}_hemi-{hemi}_bold.func.gii')        
    timeseries[i] = nib.load(filename).agg_data()
timeseries = np.vstack(timeseries) # (20484, 135)

fmriprep_confounds_file = op.join(bids_folder,'derivatives', 'fmriprep', f'sub-{sub}', f'ses-{ses}', 'func', f'sub-{sub}_ses-{ses}_task-{task}_run-{run}_desc-confounds_timeseries.tsv')
fmriprep_confounds = pd.read_table(fmriprep_confounds_file)[fmriprep_confounds_include] 
fmriprep_confounds= fmriprep_confounds.fillna(method='bfill')

In [10]:
from utils import get_events_confounds

for run in range(1,7):
    dm = get_events_confounds(sub, ses,run,bids_folder)
    #regressors_to_remove = pd.concat([dm.reset_index(drop=True), fmriprep_confounds], axis=1)

/Users/mrenke/git/numrisk/numrisk/fmri_analysis/gradients/utils.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3
/Users/mrenke/git/numrisk/numrisk/fmri_analysis/gradients/utils.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3


A 'modulation' column was found in the given events data and is used.
Design matrix created to remove task effects. shape:
(188, 12)
A 'modulation' column was found in the given events data and is used.
Design matrix created to remove task effects. shape:
(188, 12)
A 'modulation' column was found in the given events data and is used.
Design matrix created to remove task effects. shape:
(188, 12)
A 'modulation' column was found in the given events data and is used.


/Users/mrenke/git/numrisk/numrisk/fmri_analysis/gradients/utils.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3
/Users/mrenke/git/numrisk/numrisk/fmri_analysis/gradients/utils.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3


Design matrix created to remove task effects. shape:
(188, 12)
A 'modulation' column was found in the given events data and is used.
Design matrix created to remove task effects. shape:
(188, 12)
A 'modulation' column was found in the given events data and is used.


/Users/mrenke/git/numrisk/numrisk/fmri_analysis/gradients/utils.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3
/Users/mrenke/git/numrisk/numrisk/fmri_analysis/gradients/utils.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3


Design matrix created to remove task effects. shape:
(188, 12)


In [12]:
run=4
tr = 2.3 # repetition Time
n = 188 # number of slices # adjust this important!!

df_events = pd.read_csv(op.join(bids_folder, f'sub-{sub}', f'ses-{ses}', 'func', f'sub-{sub}_ses-{ses}_task-{task}_run-{run}_events.tsv'.format(sub=sub, ses=ses)), sep='\t') # before run was ot interated over (run-1)

stimulus1 = df_events.loc[df_events['trial_type'] == 'stimulus 1', ['onset', 'trial_nr', 'trial_type', 'n1']]
stimulus1['duration'] = 0.6 + 0.8
stimulus1['onset'] = stimulus1['onset'] - 0.8 # cause we want to take the onset of the piechart 
stimulus1['stim_order'] = int(1)
stimulus1_int = stimulus1.copy()
stimulus1_int['trial_type'] = 'stimulus1_int'
stimulus1_int['modulation'] = 1
stimulus1_mod= stimulus1.copy()
stimulus1_mod['trial_type'] = 'stimulus1_mod'
stimulus1_mod['modulation'] = stimulus1['n1']

#choices = df_events.xs('choice', 0, 'trial_type', drop_level=False).reset_index('trial_type')[['onset', 'trial_nr', 'trial_type', 'n2']]
choices = df_events.loc[df_events['trial_type'] == 'choice']

#stimulus2 = df_events.xs('stimulus 2', 0, 'trial_type', drop_level=False).reset_index('trial_type')[['onset', 'trial_nr', 'trial_type', 'n2']]
stimulus2 = df_events.loc[df_events['trial_type'] == 'stimulus 2', ['onset', 'trial_nr', 'trial_type', 'n2']]
stimulus2 = stimulus2.set_index('trial_nr')
stimulus2['duration'] = choices.set_index('trial_nr')['onset']- stimulus2['onset'] + 0.6 # 0.6 + 0.6 ## looked at the data, is is different for stim 1 and 2... ?!!
stimulus2['onset'] = stimulus2['onset'] - 0.6
stimulus2['stim_order'] = int(2)
stimulus2_int = stimulus2.copy()
stimulus2_int['trial_type'] = 'stimulus2_int'
stimulus2_int['modulation'] = 1
stimulus2_mod= stimulus2.copy()
stimulus2_mod['trial_type'] = 'stimulus2_mod'
stimulus2_mod['modulation'] = stimulus2['n2']

In [18]:
np.shape(stimulus1_int)

(30, 7)

In [20]:
stimulus2_int.reset_index(inplace=True)
stimulus2_mod.reset_index(inplace=True)

In [22]:
events = pd.concat((stimulus1_int,stimulus1_mod, stimulus2_int, stimulus2_mod)).set_index(['trial_nr','stim_order'],append=True).sort_index()

events

onset     trial_type    n1  duration  modulation  \
   trial_nr stim_order                                                          
0  91       1            14.031805  stimulus1_int  14.0  1.400000         1.0   
            1            14.031805  stimulus1_mod  14.0  1.400000        14.0   
            2            20.821294  stimulus2_int   NaN  1.855832         1.0   
            2            20.821294  stimulus2_mod   NaN  1.855832         4.0   
1  92       2            34.517140  stimulus2_int   NaN  1.522109         1.0   
...                            ...            ...   ...       ...         ...   
81 118      1           376.917562  stimulus1_mod  10.0  1.400000        10.0   
84 119      1           392.632114  stimulus1_int  10.0  1.400000         1.0   
            1           392.632114  stimulus1_mod  10.0  1.400000        10.0   
87 120      1           404.826760  stimulus1_int  10.0  1.400000         1.0   
            1           404.826760  stimulus1_mod  10.0  1.400000        10.0   

                          n2  
   trial_nr stim_order        
0  91       1            NaN  
            1            NaN  
            2            4.0  
            2            4.0  
1  92       2           19.0  
...                      ...  
81 118      1            NaN  
84 119      1            NaN  
            1            NaN  
87 120      1            NaN  
            1            NaN  

[120 rows x 6 columns]

In [ ]:
clean_ts = cleanTS(sub, ses,remove_task_effects, bids_folder=bids_folder) # checks if fsav5-file exists, if not, creates it
